In [ ]:
#| echo: false

import numpy as np
import pandas as pd
import requests
import scipy.stats as ss
import matplotlib as mpl
import json
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from urllib.request import urlopen
from IPython.display import Image
from IPython.display import HTML

## Methodology

The Healthy, Wealthy, and Wise indices combine multiple indicators into a single, comparative measure of economic well-being. The index is constructed from the U.S. Census Bureau's American Community Survey (ACS) 5-Year Estimates and County Business Patterns (CBP) datasets. Tables \@tbl-healthy, \@tbl-wealthy, and \@tbl-wise contain the indicators for the respective index.

The individual indicators are individually normalized across 105 counties on a scale of 0 to 100, where 100 represents the most distressed county and 0 represents the least distressed county. The individual indicators are normalized using the following formula:

$$\text{Index Indicator}_{ij} = \frac{X_{ij}-\text{min}_{(\text{of } 105)}\{X_i\}}{\text{max}_{(\text{of } 105)}\{X_i\}-\text{min}_{(\text{of } 105)}\{X_i\}} \times 100,$$

where $X_{ij}$ is the value of indicator $i$ for county $j$. The final index value results from taking the geometric mean of the respective indicators. To handle the zeros, one is added to each normalized indicator; therefore, one is then subtracted from the calculated mean.


In [ ]:
#| echo: false
#| label: fig-healthy
#| fig-cap: Healthy Indicators

fig-healthy.update_layout(margin=dict(l=50))

values = [
    [
        '% Fair or Poor Health',
        'Average Number of Physically Unhealthy Days',
        'Average Number of Mentally Unhealthy Days',
        '% Low Birthweight',
        '% Adults Reporting Currently Smoking',
        '% Adults with Obesity',
        '% Physically Inactive',
        '% Excessive Drinking',
        '% Uninsured'
    ], #1st col
    [
        'Percentage of adults that report fair or poor health',
        'Average number of reported physically unhealthy days per month',
        'Average number of reported mentally unhealthy days per month',
        'Percentage of births with low birthweight (<2500g)',
        'Percentage of adults that reported currently smoking',
        'Percentage of adults that report BMI >= 30',
        'Percentage of adults that report no leisure-time physical activity',
        'Percentage of adults that report excessive drinking',
        'Percentage of people under age 65 without insurance'
    ]
]


fig = go.Figure(data=[go.Table(
  columnorder = [1,2],
  columnwidth = [25,60],
  header = dict(
    values = [
        ['<b>Indicator</b>'],
        ['<b>Description</b>']
    ],
    line_color='darkslategray',
    fill_color=['rgb(241, 241, 241)', 'rgb(241, 241, 241)'],
    align=['center','center'],
    font=dict(color='rgb(22,74,88)', size=12),
    height=20
  ),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['white', 'white']),
    align=['left', 'left'],
    font=dict(color='black', size=12),
    height=30)
    )
])

fig.show()

In [ ]:
#| echo: false
#| label: fig-wealthy
#| fig-cap: Wealthy Indicators
values = [
    [
        'Income Ratio', 
        '% Children in Poverty', 
        'Unemployment Rate'
    ], #1st col
    [
        'Ratio of household income at the 80th percentile to income at the 20th percentile',
        'Percentage of children (under age 18) living in poverty',
        'Percentage of population ages 16+ who are unemployed'
    ]
]


fig = go.Figure(data=[go.Table(
  columnorder = [1,2],
  columnwidth = [25,60],
  header = dict(
    values = [
        ['<b>Indicator</b>'],
        ['<b>Description</b>']
    ],
    line_color='darkslategray',
    fill_color=['rgb(241, 241, 241)', 'rgb(241, 241, 241)'],
    align=['center','center'],
    font=dict(color='rgb(22,74,88)', size=12),
    height=20
  ),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['white', 'white']),
    align=['left', 'left'],
    font=dict(color='black', size=12),
    height=30)
    )
])

fig.show()

In [ ]:
#| echo: false
#| label: fig-wise
#| fig-cap: Wise Indicators

values = [
    [
        '% Completed High School', 
        '% Some College', 
        'Social Association Rate'
    ], #1st col
    [
        'Percentage of adults age 25 and over with a high school diploma or equivalent',
        'Percentage of adults age 25-44 with some post-secondary education',
        'Social associations per 10,000 population'
    ]
]


fig = go.Figure(data=[go.Table(
  columnorder = [1,2],
  columnwidth = [25,60],
  header = dict(
    values = [
        ['<b>Indicator</b>'],
        ['<b>Description</b>']
    ],
    line_color='darkslategray',
    fill_color=['rgb(241, 241, 241)', 'rgb(241, 241, 241)'],
    align=['center','center'],
    font=dict(color='rgb(22,74,88)', size=12),
    height=20
  ),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['white', 'white']),
    align=['left', 'left'],
    font=dict(color='black', size=12),
    height=30)
    )
])
fig.show()

## Results


In [ ]:
#| echo: false
#| label: fig-healthy-Index
#| fig-cap: Healthy Index

token = 'pk.eyJ1Ijoid3N1cHBtYyIsImEiOiJjbDA2dWNiZmEyYjRqM2lsc2hrZ2g5Z3ZrIn0.Qcm0AOjbrEP6NPJ6u-14EA'

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)  

df = pd.read_csv('data/healthy-wealthy-wise-data.csv', dtype={'fips':'str'})
df = df.sort_values(by=['healthy_quartile'])

colorscales = [
    ((0.0, 'rgba(22,74,88,0.25)'), (1.0, 'rgba(22,74,88,0.25)')),
    ((0.0, 'rgba(22,74,88,0.5)'), (1.0, 'rgba(22,74,88,0.5)')),
    ((0.0, 'rgba(22,74,88,0.75)'), (1.0, 'rgba(22,74,88,0.75)')),
    ((0.0, 'rgba(22,74,88,1.0)'), (1.0, 'rgba(22,74,88,1.0)'))
]

fig = go.Figure()
for i, quartile in enumerate(df['healthy_quartile'].unique()):
    dfp = df[df['healthy_quartile'] == quartile]
    fig.add_choroplethmapbox(
        geojson=counties, locations=dfp['fips'],
        z=[i,] * len(dfp), featureidkey='id',
        showlegend=True, name=quartile,
        colorscale=colorscales[i], showscale=False,
        text='County:  ' + dfp['county'].str.title() +
        '<br>Quartile:  ' + dfp['healthy_quartile'].str.split(' ').str[0],
        hoverinfo='text'
    )

fig.update_layout(legend_title='Index Performance', margin={'r':0,'t':0,'l':0,'b':0},
                  mapbox_accesstoken=token,
                  mapbox=dict(style='carto-positron', zoom=6, 
                              center={'lat': 38.4937, 'lon': -98.3804},))

fig.show()
print("\n")
print("\n")

In [ ]:
#| echo: false
#| label: fig-wealthy-Index
#| fig-cap: Wealthy Index

df = df.sort_values(by=['wealthy_quartile'])

colorscales = [
    ((0.0, 'rgba(231,38,40,0.25)'), (1.0, 'rgba(231,38,40,0.25)')),
    ((0.0, 'rgba(231,38,40,0.5)'), (1.0, 'rgba(231,38,40,0.5)')),
    ((0.0, 'rgba(231,38,40,0.75)'), (1.0, 'rgba(231,38,40,0.75)')),
    ((0.0, 'rgba(231,38,40,1.0)'), (1.0, 'rgba(231,38,40,1.0)'))
]

fig = go.Figure()
for i, quartile in enumerate(df['wealthy_quartile'].unique()):
    dfp = df[df['wealthy_quartile'] == quartile]
    fig.add_choroplethmapbox(
        geojson=counties, locations=dfp['fips'],
        z=[i,] * len(dfp), featureidkey='id',
        showlegend=True, name=quartile,
        colorscale=colorscales[i], showscale=False,
        text='County:  ' + dfp['county'].str.title() +
        '<br>Quartile:  ' + dfp['wealthy_quartile'].str.split(' ').str[0],
        hoverinfo='text'
    )

fig.update_layout(legend_title='Index Performance', margin={'r':0,'t':0,'l':0,'b':0},
                  mapbox_accesstoken=token,
                  mapbox=dict(style='carto-positron', zoom=6, 
                              center={'lat': 38.4937, 'lon': -98.3804},))

fig.show()

print("\n")
print("\n")

In [ ]:
#| echo: false
#| label: fig-wise-Index
#| fig-cap: Wise Index

df = df.sort_values(by=['wise_quartile'])

colorscales = [
    ((0.0, 'rgba(255,193,0,0.25)'), (1.0, 'rgba(255,193,0,0.25)')),
    ((0.0, 'rgba(255,193,0,0.5)'), (1.0, 'rgba(255,193,0,0.5)')),
    ((0.0, 'rgba(255,193,0,0.75)'), (1.0, 'rgba(255,193,0,0.75)')),
    ((0.0, 'rgba(255,193,0,1.0)'), (1.0, 'rgba(255,193,0,1.0)'))
]

fig = go.Figure()
for i, quartile in enumerate(df['wise_quartile'].unique()):
    dfp = df[df['wise_quartile'] == quartile]
    fig.add_choroplethmapbox(
        geojson=counties, locations=dfp['fips'],
        z=[i,] * len(dfp), featureidkey='id',
        showlegend=True, name=quartile,
        colorscale=colorscales[i], showscale=False,
        text='County:  ' + dfp['county'].str.title() +
        '<br>Quartile:  ' + dfp['wise_quartile'].str.split(' ').str[0],
        hoverinfo='text'
    )

fig.update_layout(legend_title='Index Performance', margin={'r':0,'t':0,'l':0,'b':0},
                  mapbox_accesstoken=token,
                  mapbox=dict(style='carto-positron', zoom=6, 
                              center={'lat': 38.4937, 'lon': -98.3804},))

fig.show()